In [1]:
import cv2
import dlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import distance as dist

from io import BytesIO
from IPython.display import clear_output, Image, display
from PIL import Image as Img

In [2]:
def padronizar_imagem(imagem):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
    imagem = cv2.resize(imagem, (500, 400))
    return imagem

def exibir_video(frame):
    img = Img.fromarray(frame, "RGB")
    buffer = BytesIO()
    img.save(buffer, format="JPEG")
    display(Image(data=buffer.getvalue()))
    clear_output(wait=True)

In [3]:
classificador_dlib_68_pontos = "data/classificadores/shape_predictor_68_face_landmarks.dat"
# download: https://github.com/italojs/facial-landmarks-recognition/blob/master/shape_predictor_68_face_landmarks.dat
classificador_dlib = dlib.shape_predictor(classificador_dlib_68_pontos)
detector_face = dlib.get_frontal_face_detector()

In [4]:
FACE = list(range(17, 68))
FACE_COMPLETA = list(range(0, 68))
LABIO = list(range(48, 61))
SOMBRANCELHA_DIREITA = list(range(17, 22))
SOMBRANCELHA_ESQUERDA = list(range(22, 27))
OLHO_DIREITO = list(range(36, 42))
OLHO_ESQUERDO = list(range(42, 48))
NARIZ = list(range(27, 35))
MANDIBULA = list(range(0, 17))

In [5]:
def pontos_marcos_faciais(imagem):
    retangulos = detector_face(imagem, 1)

    if len(retangulos) == 0:
        return None

    marcos = []

    for retangulo in retangulos:
        marcos.append(np.matrix([[p.x, p.y] for p in classificador_dlib(imagem, retangulo).parts()]))

    return marcos

In [6]:
def anotar_marcos_faciais(imagem, marcos_faciais):
    if imagem is None:
        return im

    for marco in marcos_faciais:
        for idx, ponto in enumerate(marco):
            centro = (ponto[0, 0], ponto[0, 1])
            cv2.circle(imagem, centro, 3, color=(255, 255, 0), thickness=-1)
    return imagem

In [7]:
def aspecto_razao_boca(pontos_boca):
    a = dist.euclidean(pontos_boca[3].tolist()[0], pontos_boca[9].tolist()[0])
    b = dist.euclidean(pontos_boca[2].tolist()[0], pontos_boca[10].tolist()[0])
    c = dist.euclidean(pontos_boca[4].tolist()[0], pontos_boca[8].tolist()[0])
    d = dist.euclidean(pontos_boca[0].tolist()[0], pontos_boca[6].tolist()[0])
    aspecto_razao = (a + b + c)/(3.0 * d)
    return aspecto_razao

In [9]:
captura_video = cv2.VideoCapture(0)

try:
    aspecto_razao_max = 0
    qtde_bocejo = 0

    bocejo = False
    bocejo_anterior = False

    while(True):
        captura_ok, frame = captura_video.read()

        if captura_ok:
            frame = padronizar_imagem(frame)
            pontos = pontos_marcos_faciais(frame)

            if pontos is not None:

                aspecto_razao = aspecto_razao_boca(pontos[0][LABIO])
                aspecto_razao = round(aspecto_razao, 3)

                if aspecto_razao > aspecto_razao_max:
                    aspecto_razao_max = aspecto_razao

                aspecto_razao_info = "aspecto razao " + str(aspecto_razao) + " maximo " + str(aspecto_razao_max)

                frame = anotar_marcos_faciais(frame, pontos)

                coord = tuple(pontos[0][LABIO][0].A1.reshape(1, -1)[0])
                coord = (coord[0], coord[1] + 20)

                cv2.putText(frame, aspecto_razao_info, coord, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)

                if aspecto_razao > 0.6:
                    bocejo = True
                else:
                    bocejo = False

                if bocejo_anterior == False and bocejo == True:
                    qtde_bocejo += 1

                coord = (coord[0], coord[1] + 23)
                cv2.putText(frame, "bocejos " + str(qtde_bocejo), coord, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

                bocejo_anterior = bocejo

            exibir_video(frame)

except KeyboardInterrupt:
    captura_video.release()
    print("Interrompido")

Interrompido
